In [ ]:
!wget https://www.dropbox.com/s/n30i1tiwi3s64fg/X-Ray_B.zip?dl=0

In [ ]:
!unzip X-Ray_B.zip?dl=0

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import ResNet50,VGG16,MobileNet
from tensorflow.keras.applications.resnet50 import preprocess_input

In [4]:
train_dir = "X-Ray/train/"
valid_dir = "X-Ray/test/"

img_width, img_height = 224, 224

##VGG16

In [5]:
conv_base = VGG16(weights='imagenet', 
                  include_top=False,
                  input_shape=(img_width, img_height, 3))

conv_base.summary()

58900480/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

## Features Extraction

In [6]:
import os, shutil
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 32

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 7, 7, 512))  
    # Must be equal to the output of the convolutional base
    labels = np.zeros(shape=(sample_count,))
    # Preprocess data
    generator = datagen.flow_from_directory(directory,
                                            target_size=(img_width,img_height),
                                            batch_size = batch_size,
                                            class_mode='binary')
    
    # Pass data through convolutional base
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size: (i + 1) * batch_size] = features_batch
        labels[i * batch_size: (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
    return features, labels
    
train_features, train_labels = extract_features(train_dir, 1046)  
validation_features, validation_labels = extract_features(valid_dir, 466)

Found 1046 images belonging to 2 classes.
Found 466 images belonging to 2 classes.


In [7]:
train_labels

array([1., 0., 1., ..., 0., 1., 1.])

In [8]:
nsamples1, nx1, ny1 ,nz1= train_features.shape
d2_train_dataset = train_features.reshape((nsamples1,nx1*ny1*nz1))

In [9]:
nsamples, nx, ny ,nz= validation_features.shape
d2_test_dataset = validation_features.reshape((nsamples,nx*ny*nz))

### Implementing SVM

In [10]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
import numpy as np

In [11]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

svc=SVC() 
svc.fit(d2_train_dataset,train_labels)

y_pred=svc.predict(d2_test_dataset)

In [ ]:
#print('Model accuracy score with default hyperparameters: {0:0.4f}' . format(accuracy_score(validation_labels, y_pred)))
print('Model accuracy score with default hyperparameters: {0:0.4f} : ')

Model accuracy score with default hyperparameters: {0:0.4f} : 


In [12]:
print('Model accuracy score with default hyperparameters: {0:0.4f}' . format(accuracy_score(validation_labels, y_pred)))

Model accuracy score with default hyperparameters: 0.9485


In [13]:
accuracy_score(y_pred,validation_labels)
print(classification_report(y_pred,validation_labels))

              precision    recall  f1-score   support

         0.0       0.90      1.00      0.95       209
         1.0       1.00      0.91      0.95       257

    accuracy                           0.95       466
   macro avg       0.95      0.95      0.95       466
weighted avg       0.95      0.95      0.95       466



In [14]:
confusion_matrix(y_pred,validation_labels)

array([[209,   0],
       [ 24, 233]])

### Implementing a Random Forrest

In [15]:
from sklearn.ensemble import RandomForestClassifier

In [16]:
newmodel=RandomForestClassifier()

In [17]:
newmodel.fit(d2_train_dataset,train_labels)

RandomForestClassifier()

In [18]:
y_pred=newmodel.predict(d2_test_dataset)
y_pred

array([1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1.,
       0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1.,
       1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1., 0., 1., 0., 0.,
       1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 1., 0., 1., 1., 0., 1.,
       1., 0., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1.,
       0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0.,
       1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1.,
       1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0.,
       0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0.,
       0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1.,
       1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1.,
       0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 1., 1., 0.,
       0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1.,
       1., 0., 0., 1., 1.

In [19]:
accuracy_score(y_pred,validation_labels)
print(classification_report(y_pred,validation_labels))

              precision    recall  f1-score   support

         0.0       0.95      0.97      0.96       228
         1.0       0.97      0.95      0.96       238

    accuracy                           0.96       466
   macro avg       0.96      0.96      0.96       466
weighted avg       0.96      0.96      0.96       466



In [20]:
confusion_matrix(y_pred,validation_labels)

array([[221,   7],
       [ 12, 226]])

### Implementing a KNN

In [21]:
from sklearn.neighbors import KNeighborsClassifier

In [22]:
knn=KNeighborsClassifier(n_neighbors=7)

In [23]:
knn.fit(d2_train_dataset,train_labels)

KNeighborsClassifier(n_neighbors=7)

In [24]:
y_pred_knn=knn.predict(d2_test_dataset)
y_pred_knn

array([0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1.,
       0., 0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1.,
       1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1., 0., 1., 0., 0.,
       1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 1., 0., 1., 1., 0., 1.,
       1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1.,
       0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0.,
       1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1.,
       1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0.,
       0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0.,
       0., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1.,
       1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1.,
       0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0.,
       0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
       1., 1., 0., 1., 1.

In [25]:
accuracy_score(y_pred_knn,validation_labels)
print(classification_report(y_pred_knn,validation_labels))

              precision    recall  f1-score   support

         0.0       0.93      0.96      0.94       226
         1.0       0.96      0.93      0.94       240

    accuracy                           0.94       466
   macro avg       0.94      0.94      0.94       466
weighted avg       0.94      0.94      0.94       466



In [26]:
confusion_matrix(y_pred_knn,validation_labels)

array([[216,  10],
       [ 17, 223]])

## Implementing a Decision Tree

In [27]:
from sklearn.tree import DecisionTreeClassifier

In [28]:
dtc=DecisionTreeClassifier()

In [29]:
dtc.fit(d2_train_dataset,train_labels)

DecisionTreeClassifier()

In [30]:
y_pred_dtc=dtc.predict(d2_test_dataset)

In [31]:
y_pred_dtc

array([0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1.,
       0., 0., 1., 1., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1.,
       1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0.,
       1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0.,
       1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1.,
       0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0.,
       1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1.,
       1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1., 0.,
       0., 1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0.,
       1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 1.,
       0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1.,
       0., 0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1.,
       0., 0., 0., 1., 1.

In [32]:
accuracy_score(y_pred_dtc,validation_labels)
print(classification_report(y_pred_dtc,validation_labels))

              precision    recall  f1-score   support

         0.0       0.91      0.84      0.87       252
         1.0       0.82      0.90      0.86       214

    accuracy                           0.86       466
   macro avg       0.86      0.87      0.86       466
weighted avg       0.87      0.86      0.87       466



In [33]:
confusion_matrix(y_pred_dtc,validation_labels)

array([[211,  41],
       [ 22, 192]])

## Implementing Gaussian Naive Bayes

In [34]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(d2_train_dataset,train_labels)

GaussianNB()

In [35]:
y_pred=dtc.predict(d2_test_dataset)

In [36]:
accuracy_score(y_pred,validation_labels)
print(classification_report(y_pred,validation_labels))

              precision    recall  f1-score   support

         0.0       0.91      0.84      0.87       252
         1.0       0.82      0.90      0.86       214

    accuracy                           0.86       466
   macro avg       0.86      0.87      0.86       466
weighted avg       0.87      0.86      0.87       466



## Implementing AdaBoost

In [37]:
from sklearn.ensemble import AdaBoostClassifier

In [38]:
classifier = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1),
    n_estimators=200
)
classifier.fit(d2_train_dataset, train_labels)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                   n_estimators=200)

In [39]:
predictions = classifier.predict(d2_test_dataset)

In [40]:
accuracy_score(predictions,validation_labels)
print(classification_report(predictions,validation_labels))

              precision    recall  f1-score   support

         0.0       0.97      0.97      0.97       233
         1.0       0.97      0.97      0.97       233

    accuracy                           0.97       466
   macro avg       0.97      0.97      0.97       466
weighted avg       0.97      0.97      0.97       466

